**Matan Leventer - 208447029**

**Din Ezra - 206065989**

**Guy Yutal - 308013861**

In [ ]:
!pip install catboost

In [ ]:
import pickle
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from scipy.stats import zscore


In [ ]:
# see more details in the course first lecture
!mkdir /root/.kaggle/
import json
import os

# Installing the Kaggle package
!pip install kaggle

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"matanleventer","key":"b8b0b970f31143ccc2f842e4fce4b3b9"}


# creating kaggle.json file with the personal API-Key details
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import kaggle
!mkdir ./datasets

# download the dataset from Kaggle and unzip it
# !kaggle datasets download fuzzyfroghunter/dickens -p ./datasets/
# !unzip ./datasets/dickens.zip  -d ./datasets/

!kaggle competitions download -c recommendation-systems-click-through-rate -p ./datasets/

# Make sure to replace 'dickens.zip' with the actual name of the downloaded file
!unzip ./datasets/recommendation-systems-click-through-rate.zip -d ./datasets/

100% 11.7G/11.7G [02:04<00:00, 133MB/s]
100% 11.7G/11.7G [02:04<00:00, 101MB/s]
Archive:  ./datasets/recommendation-systems-click-through-rate.zip
  inflating: ./datasets/Taboola Dataset Notice.docx  
  inflating: ./datasets/submission_example.csv  
  inflating: ./datasets/test_file.csv  
  inflating: ./datasets/train_data/train1/part-00000.csv  
  inflating: ./datasets/train_data/train1/part-00001.csv  
  inflating: ./datasets/train_data/train1/part-00002.csv  
  inflating: ./datasets/train_data/train1/part-00003.csv  
  inflating: ./datasets/train_data/train1/part-00004.csv  
  inflating: ./datasets/train_data/train1/part-00005.csv  
  inflating: ./datasets/train_data/train1/part-00006.csv  
  inflating: ./datasets/train_data/train1/part-00007.csv  
  inflating: ./datasets/train_data/train1/part-00008.csv  
  inflating: ./datasets/train_data/train1/part-00009.csv  
  inflating: ./datasets/train_data/train1/part-00010.csv  
  inflating: ./datasets/train_data/train2/part-00011.csv  
  

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_folder):
        self.root_folder = root_folder
        self.folds = sorted(os.listdir(root_folder))

    def __len__(self):
        return len(self.folds)



    def __getitem__(self, fold_idx):
      fold_path = os.path.join(self.root_folder, self.folds[fold_idx])
      files = sorted(os.listdir(fold_path))
      # files = files[until_csv:until_csv+3]
      # Read all CSV files at once
      concatenated_data_list = []
      r=0
      for selected_file in files:
        if r<8:
          csv_file_path = os.path.join(fold_path, selected_file)
          df = pd.read_csv(csv_file_path)
          concatenated_data_list.append(df)
        else:
          break
        r+=1



      # Concatenate the data from all CSV files
      concatenated_data = pd.concat(concatenated_data_list, ignore_index=True)

      return concatenated_data

def save_model(model, filename):
    # Save the model state to a file using pickle
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

def load_model(filename):
    # Load the model state from a file using pickle
    with open(filename, 'rb') as file:
        loaded_model = pickle.load(file)
    return loaded_model



In [ ]:
def Graphs(df):
  plt.figure(figsize=(70,20))
  sns.countplot(x='time_of_day',hue='is_click',data=df)
  plt.figure(figsize=(70,20))
  sns.countplot(x='gmt_offset',hue='is_click',data=df)
  plt.figure(figsize=(70,20))
  sns.countplot(x='day_of_week',hue='is_click',data=df)

In [ ]:
def Corr(df):
  corr = df.corr()
  f, ax = plt.subplots(figsize=(10,10))
  sns.heatmap(corr, ax=ax, annot=True,linewidths=3,cmap='YlGn')
  plt.title("Pearson correlation of Features", y=1.05, size=15)

In [ ]:
def label_hour(hour):
    return 'A' if 8 <= hour <= 17 else 'B'

def label_day(hour):
    return 'A' if 1 <= hour <= 4 else 'B'

In [ ]:
def distribution_value(df):
  for column_name in df.columns:
      # Check number of unique values
      unique_values = df[column_name].nunique()

      # Check value counts
      value_counts = df[column_name].value_counts()

      print(f'Column: {column_name}')
      print(f'Number of unique values: {unique_values}')
      print(f'Value counts:\n{value_counts}\n')


In [ ]:
def drop_col(df):
    list_drop = ['Id']
    target1 = df['is_click']
    target2 = df['Id']

    df = df.drop(columns=list_drop)
    return df,target1,target2

In [ ]:
def normlize(df):
  list_norm = ['page_view_start_time','user_recs','user_clicks','user_target_recs','empiric_calibrated_recs','empiric_clicks','user_recs_per_click']
  # list_dummies = ['new_time_of_day','new_day_of_week']
  list_drop = ['user_id_hash','gmt_offset']
  scaler = StandardScaler()
  encoder = LabelEncoder()

  categorical_columns = ['target_id_hash', 'syndicator_id_hash', 'campaign_id_hash', 'target_item_taxonomy', 'placement_id_hash', 'publisher_id_hash', 'source_id_hash', 'source_item_type', 'browser_platform', 'os_family', 'country_code', 'region', 'day_of_week', 'time_of_day']


  df['region'].fillna('Unknown', inplace=True)
  df['country_code'].fillna('Unknown', inplace=True)

  # df['time_of_day'] = df['time_of_day'].astype(str)
  # df['os_family'] = df['os_family'].astype(str)
  # df['day_of_week'] = df['day_of_week'].astype(str)
  # df['gmt_offset'] = df['gmt_offset'].astype(str)

  df['user_recs_per_click'] = df['user_recs'] / (df['user_clicks'] + 1)

  df = df.drop(columns=list_drop)

  for col in categorical_columns:
    df[col] = df[col].astype('category')

  # df = pd.get_dummies(df, columns=list_dummies)
  df[list_norm] = scaler.fit_transform(df[list_norm])
  return df


In [ ]:
def split_dataframe(df,test):
  concatenated_data_list = []
  concatenated_data = pd.concat([df,test], ignore_index=True)

  concatenated_data,target1,target2 = drop_col(concatenated_data)
  concatenated_data = normlize(concatenated_data)
  first_column = concatenated_data.pop(concatenated_data.columns[0])

  concatenated_data.insert(len(concatenated_data.columns), first_column.name, first_column)
  df_train= concatenated_data[concatenated_data['is_click'].notna()]
  target11= target1.dropna()

  df_test = concatenated_data[concatenated_data['is_click'].isna()]

  df_train = df_train.drop(columns='is_click')
  df_test = df_test.drop(columns='is_click')

  return df_train,df_test,target11,

In [ ]:
# Graphs(df)

In [ ]:
# Corr(df)

In [ ]:
data_root = '/content/datasets/train_data'

# # Create an instance of your custom dataset
custom_dataset = CustomDataset(root_folder=data_root)

df = custom_dataset.__getitem__(0)
test = pd.read_csv('/content/datasets/test_file.csv')

In [ ]:
X,test,y = split_dataframe(df,test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = CatBoostClassifier(iterations=400, depth=6, learning_rate=0.1, loss_function='Logloss', eval_metric='AUC')

cat_features = ['target_id_hash',
      'syndicator_id_hash', 'campaign_id_hash','target_item_taxonomy', 'placement_id_hash','publisher_id_hash',
      'source_id_hash', 'source_item_type', 'browser_platform','country_code','region',
        'os_family','day_of_week','time_of_day']

model.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_test, y_test), verbose=10)

0:	test: 0.8279014	best: 0.8279014 (0)	total: 15.9s	remaining: 1h 45m 52s
10:	test: 0.8422006	best: 0.8422006 (10)	total: 2m 16s	remaining: 1h 20m 11s
20:	test: 0.8470976	best: 0.8470976 (20)	total: 4m 43s	remaining: 1h 25m 22s
30:	test: 0.8491654	best: 0.8491654 (30)	total: 6m 58s	remaining: 1h 23m 1s
40:	test: 0.8506650	best: 0.8506650 (40)	total: 9m 18s	remaining: 1h 21m 29s
50:	test: 0.8516922	best: 0.8516922 (50)	total: 11m 28s	remaining: 1h 18m 28s
60:	test: 0.8523585	best: 0.8523585 (60)	total: 13m 42s	remaining: 1h 16m 11s
70:	test: 0.8529393	best: 0.8529393 (70)	total: 15m 53s	remaining: 1h 13m 36s
80:	test: 0.8534231	best: 0.8534231 (80)	total: 18m 10s	remaining: 1h 11m 33s
90:	test: 0.8537813	best: 0.8537813 (90)	total: 20m 35s	remaining: 1h 9m 55s
100:	test: 0.8540522	best: 0.8540522 (100)	total: 22m 53s	remaining: 1h 7m 45s
110:	test: 0.8543431	best: 0.8543431 (110)	total: 25m 10s	remaining: 1h 5m 32s
120:	test: 0.8545739	best: 0.8545739 (120)	total: 27m 26s	remaining: 1h 

In [ ]:
feature_importances = model.feature_importances_
sorted_importances = sorted(zip(X_train.columns, feature_importances), key=lambda x: x[1], reverse=True)

# Print feature importances
for feature, importance in sorted_importances:
    print(f"{feature}: {importance}")

placement_id_hash: 28.817272306140065
syndicator_id_hash: 14.287162611994459
user_clicks: 14.167767277644225
user_recs_per_click: 9.567106915112216
campaign_id_hash: 8.924924211600826
user_recs: 5.400145610729701
publisher_id_hash: 4.0631025581115
browser_platform: 2.900324585441087
source_item_type: 1.6937088057632448
os_family: 1.5290955349470383
source_id_hash: 1.38828663042741
target_id_hash: 1.2388893280630358
empiric_clicks: 1.2101745203699419
user_target_recs: 1.0145423040141879
empiric_calibrated_recs: 0.9942744674034483
target_item_taxonomy: 0.924302938771808
day_of_week: 0.698482818097336
page_view_start_time: 0.4361795138721836
region: 0.3631559640037618
time_of_day: 0.20706853973371056
country_code: 0.1740325577587312


In [ ]:
clf = xgb.XGBClassifier(tree_method="hist", enable_categorical=True)
clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
# Define parameters
params = {
    'objective': 'binary',  # or 'multiclass' for multiclass classification
    'metric': 'binary_logloss',  # or other evaluation metrics
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_estimators': 100,
    'boosting_type': 'gbdt',  # Gradient Boosting Decision Tree
    'random_state': 42
}

# Create and train the model
lgb_model = lgb.LGBMClassifier(**params)
lgb_model.fit(X_train, y_train)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 1420219, number of negative: 1419763
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.748197 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23118
[LightGBM] [Info] Number of data points in the train set: 2839982, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500080 -> initscore=0.000321
[LightGBM] [Info] Start training from score 0.000321


LGBMClassifier(colsample_bytree=0.8, metric='binary_logloss',
               objective='binary', random_state=42, subsample=0.8)

In [ ]:
predictions_a = clf.predict_proba(test)
predictions_b = model.predict_proba(test)
predictions_c = lgb_model.predict_proba(test)

predictions = (predictions_a[:,1] + predictions_b[:,1] +  predictions_c[:,1]) /3

In [ ]:
ids = np.arange(test.shape[0])

In [ ]:
new_df = pd.DataFrame({
    'Id': ids ,
    'Predicted': predictions
})

In [ ]:
new_df.to_csv('Predicted_Taboola_CTR.csv', index=False)